# Cross Validation Index Generation

In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut, StratifiedKFold
import pickle
import joblib
import os

## Load data

In [4]:
df_config = pd.read_csv('../data/config.csv')
train_id = 0
df_config

,hash_id,label,wav_files_path,processed_file_folder,processed_file_path,wav_files_info,cv_alg,cv_folds,cv_path,preproc_alg,...,pipeline_path,scaler_alg,train_data_path,train_trgt_path,target_label_file,target_id_file,model_path,model_inits,model_neurons,model_status
0,3359727688101666398,ShipsEar NN Classification,../data/shipsEar_AUDIOS,../data,../data/3359727688101666398_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,Lofar,...,../data/pipelines,StandardScaler,../data/3359727688101666398_train_data.csv,../data/3359727688101666398_trgt_data.csv,../data/models/train_label_file.csv,../data/models/train_id_file.csv,../data/models,1,../data/models/3359727688101666398_hidden_neur...,../data/models/3359727688101666398_model_statu...


### Load train data

In [5]:
df_train = pd.read_csv(df_config['train_data_path'][train_id])

In [6]:
df_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,target
0,-593.984651,75.465077,33.994589,9.742745,6.304904,4.062274,-4.479960,-12.025640,-13.893243,-10.707881,...,2.772886,4.079980,1.589406,1.082082,3.737450,5.702034,4.425509,0.643910,-3.701610,1
1,-564.030604,99.217798,30.917974,5.678145,13.161791,10.325374,-5.327357,-13.766686,-10.004396,-1.441678,...,10.295145,11.006458,7.362272,3.057871,2.907891,4.579520,3.148390,-0.558448,-4.547724,1
2,-565.570117,97.768627,28.350012,-2.708773,-1.175778,-3.744399,-13.164265,-16.510932,-13.678804,-9.134033,...,-4.559905,-1.560540,3.407577,2.858186,-2.977739,-5.271391,-3.584360,-3.995161,-6.621560,1
3,-574.731085,89.236715,27.436792,1.816695,10.781130,17.176567,6.668042,-10.990568,-21.065106,-14.921798,...,12.830178,16.924017,15.915613,10.503721,4.520189,3.524796,4.321097,0.906166,-3.552916,1
4,-574.807084,96.936057,42.420398,7.884993,4.932927,9.485642,2.500431,-10.264076,-14.679812,-7.856754,...,9.196721,12.108410,12.054569,9.160051,5.082737,2.006297,-0.259070,-1.882971,-1.695872,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579666,-129.211622,103.735884,-4.451267,13.362107,0.403481,16.740188,2.191302,-10.607187,-5.056585,9.556094,...,-4.792779,-10.805321,6.906666,-2.582687,-8.632170,-2.974505,-10.848904,-9.750429,-1.552197,10
579667,-126.285006,99.152346,-9.378488,4.103901,-10.917993,9.898384,3.995455,-4.722410,-9.989188,5.963095,...,-4.501935,-14.456355,-2.633846,-12.984082,-11.841886,-5.052002,-13.695508,-5.571090,5.285179,10
579668,-135.749433,98.374239,2.772493,20.607540,-1.142374,5.856288,-2.922496,-4.174614,-5.148741,9.535130,...,-6.730832,-15.778987,-1.742991,-9.774796,-2.168072,8.606211,-9.956886,-11.800741,1.299773,10
579669,-147.588836,95.654410,-0.590526,29.047297,-1.389854,7.434825,-1.275909,-5.044168,-4.136681,-3.874674,...,-6.146726,-20.977481,-0.186768,-1.316611,-1.148685,1.789382,-13.106041,-20.388461,-12.065845,10


# Stratified K-Folds

In [8]:
from sklearn.model_selection import StratifiedKFold
import pickle
import os

file_path = df_config['cv_path'][train_id]
n_folds = df_config['cv_folds'][train_id]

cv = StratifiedKFold(n_splits=n_folds)

data = df_train.drop(columns=['target']).values
target = df_train['target'].values

for idx, (trn_idx, val_idx) in enumerate(cv.split(data,target)):
    file_name = '%s_StratifiedKFoldsCV_fold_%i_of_%i_indexes.pkl'%(df_config['hash_id'][train_id],idx, n_folds)
    with open(os.path.join(file_path,file_name),'wb') as file_handler:
        pickle.dump([trn_idx,val_idx],file_handler)

In [18]:
data.shape

(579671, 20)

In [19]:
target.shape

(579671,)